In [ ]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('../')

# Load Student Data

In [ ]:
from src.d01_data.student_data_api import StudentDataApi

student_data_file = "/share/data/school_choice_equity/simulator_data/student/drop_optout_{}.csv"
period = "1819"
student_df = pd.read_csv(student_data_file.format(period)).set_index('studentno')
mask = student_df['grade'] == 'KG'
student_df = student_df.loc[mask]

student_df.head()

# Load Assignment Results

In [ ]:
equity_tiebreaker = 'ctip1'
diversity_category_col = 'Diversity_Category3'
program_cutoff = 'program_cutoff'

def get_specific_program_cutoff(x, diversity_category):
    x_list = x[1:-1].split()
    return float(x_list[diversity_category])

def check_tiebreaker(row):
    diversity_category = row[diversity_category_col]
    cut_off = get_specific_program_cutoff(row[program_cutoff], diversity_category)
    if cut_off > 4:
        return "sibiling"
    elif cut_off > 2 :
        return "equity"
    elif cut_off > 1:
        return "zone"
    else:
        return "lottery"


In [ ]:
assignments_dir = "/share/data/school_choice_equity/local_runs/Assignments/"
filename_template = "Assignment_{}_CTIP1_round_merged123_policyMedium1-GuardRails0-prefLength07_tiesSTB_prefExtend0_iteration{}.csv"

iteration = 0
filename = filename_template.format(equity_tiebreaker, iteration)
assignment_df = pd.read_csv(assignments_dir+trial+filename).set_index('studentno')

    
assignment_df['tiebreaker'] = assignment_df.apply(lambda row: check_tiebreaker(row), axis=1, raw=False)
assignment_df[equity_tiebreaker] = student_df[equity_tiebreaker].reindex(assignment_df.index)
assignment_df['focal'] = student_df['focal'].reindex(assignment_df.index)
assignment_df.head()

In [ ]:
evaluation_columns = [diversity_category_col, 'rank', 'designation', 'In-Zone Rank', 'tiebreaker', equity_tiebreaker, 'focal']
def q1(x):
    return np.quantile(x, .25)
def q2(x):
    return np.quantile(x, .5)
def q3(x):
    return np.quantile(x, .75)
    

assignment_df[evaluation_columns].groupby([diversity_category_col, 'focal', 'tiebreaker']).agg({'rank':['count','mean', 'min', q1, q2, q3, 'max'],
                                                                                            equity_tiebreaker: 'mean'})